## Simulator (fleet of uavs)

This notebook serves to get familiar with the simulator

In [1]:
from config import Color
from helpers import clean
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)

clean()

## Simulation Configuration

We can easily extend the configuration from Notebook 2 to multiple UAVs. 
We are also adding here a mission delay in secons.

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

base_homes= ENUPose.list([  # east, north, up, heading
    (0., 15., 0., 0.),
    (15., 0., 0., 0),
    (5., -20., 0., 30.),
    (-15., -15., 0., 0),
    (-15., 0., 0., 45),

])
base_paths = [Plan.create_square_path(side_len=10, alt=5,heading=0) for _ in base_homes]

colors=[
    Color.BLUE,
    Color.GREEN,
    Color.BLACK,
    Color.ORANGE,
    Color.RED,
]
msn_delays=[0,1,2,3,4] # in seconds

## Visualizer

Extending the visualizer configuration to multiple UAVs is also straightforward.

## Gazebo

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
gaz_config.show()

gaz= Gazebo(gaz_config,gra_origin)

## QGroundControl

In [ ]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

qgc=QGC(qgc_config)

## No Visualizer

In [5]:
novis_config = ConfigNovis(origin=gra_origin)
for home in base_homes:
    novis_config.add(base_home=home)
    
novis=NoVisualizer(novis_config)

## Launch Simulation

**Here we configure the simulation:**
* **visualizers**: Select the desired visualizer for the simulation (only one visualizer at a time is currently supported).  
* **gcs_names**: List of GCS names.  
* **missions**: List of missions for each UAV. Although advanced, customized missions can be built using the `Mission` class defined in the QGroundControl configuration script, here we use the QGroundControl configuration builder for simplicity.  
* **terminals**: Processes to display in the terminal. The available processes are defined in the `sim.py` script: `["launcher", "veh", "logic", "proxy", "gcs"]`.  
* **suppress_output**: Processes whose output should be suppressed. The available processes are defined in the `sim.py` script: `["launcher", "veh", "logic", "proxy", "gcs"]`.  
* **verbose**: Verbosity level (0 = no information, 1 = normal information, 2 = include debugging information).  
* **other arguments**: More advanced options that allow further customization.  


In [6]:
simulator = Simulator(
	visualizers=[gaz],
    gcs_names=['multicolor'],
	gcs_sysids=[[1,2,3,4,5]],
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)
orac = simulator.launch()

17:28:05 - Oracle ⚪ - INFO - 🖥️  Gazebo launched for realistic simulation and 3D visualization.
17:28:05 - Oracle ⚪ - INFO - 🚀 GCS multicolor launched (PID 357742)
17:28:05 - Oracle ⚪ - INFO - ⏳ Waiting for GCS processes to launch vehicles...
17:28:10 - Oracle ⚪ - INFO - 🔗 Starting Oracle connections to vehicles...
17:28:10 - Oracle ⚪ - INFO - 🔗 UAV logic 2 is connected to Oracle ⚪
17:28:10 - Oracle ⚪ - INFO - 🔗 UAV logic 1 is connected to Oracle ⚪
17:28:10 - Oracle ⚪ - INFO - 🔗 UAV logic 4 is connected to Oracle ⚪
17:28:10 - Oracle ⚪ - INFO - 🔗 UAV logic 3 is connected to Oracle ⚪
17:28:10 - Oracle ⚪ - INFO - 🔗 UAV logic 5 is connected to Oracle ⚪


## Oracle checking

In [7]:
orac.run()

17:28:11 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 5 vehicles
17:28:11 - Oracle ⚪ - INFO - Monitoring 1 GCS processes
17:29:35 - Oracle ⚪ - INFO - Received message 'DONE' from GCS multicolor
17:29:35 - Oracle ⚪ - INFO - GCS multicolor removal complete. Remaining connections: 0
17:29:35 - Oracle ⚪ - INFO - ✅ Main monitoring loop completed - all connections closed
17:29:36 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
